## The Traveling Salesman Problem:

* Find a tour of a given set of cities so that -
 - Each city is visited only once
 - The total distance travelled is minimized


## Import libraries

In [32]:
import csv
import numpy as np
import pandas as pd
import random # library to generate random numbers
np.random.seed(seed=42)
import math

#### Import data

In [33]:
distance_matrix = pd.read_csv("cities_and_distances.csv",header=0,index_col=0)
distance_matrix

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


#### Starting Point is Bangalore

In [34]:
starting_point = "Bangalore"

##### The number of possible solutions are: 10! = 3628800
We can generate all possible solutions and try to find the best of those , but this is a time taking process and may not be feasible for big problems.
Genetic algorithm approach takes a randomized search approach to find the best solution, with enough number of iterations, mutation and cross over, GA can reach the global minima for most of the problems.

In [35]:
possible_solutions = math.factorial(10)
print(possible_solutions)

3628800


### Cities Mapping
#### Map places to integers

In [36]:
distance_matrix.columns

Index(['Bangalore', 'Jaipur', 'Varanasi', 'Udaipur', 'Delhi', 'Chennai',
       'Mysore', 'Agra', 'Kolkata', 'Mumbai', 'Hyderabad'],
      dtype='object')

In [37]:
cities_mapping = {}

for i in enumerate(distance_matrix.columns):
    print(i[0])
    print(i[1])
    cities_mapping[i[0]] = i[1]

0
Bangalore
1
Jaipur
2
Varanasi
3
Udaipur
4
Delhi
5
Chennai
6
Mysore
7
Agra
8
Kolkata
9
Mumbai
10
Hyderabad


In [38]:
(cities_mapping)

{0: 'Bangalore',
 1: 'Jaipur',
 2: 'Varanasi',
 3: 'Udaipur',
 4: 'Delhi',
 5: 'Chennai',
 6: 'Mysore',
 7: 'Agra',
 8: 'Kolkata',
 9: 'Mumbai',
 10: 'Hyderabad'}

In [39]:
# Example
one_route = [6, 2, 10, 9, 5, 4, 7, 3, 8, 1] # one of the routes

In [40]:
one_route

[6, 2, 10, 9, 5, 4, 7, 3, 8, 1]

## createRandomRoute:: this function to generate random route

In [41]:
# Create the route with the name of cities
# Source is already fixed - Bangalore
# random.sample() - Chooses random elements from a set, used for random sampling without replacement.

def createRandomRoute():
    """ This function generates a route by sampling numbers """
    random_route = random.sample(range(1,11), 10)    # excluding 0 from the range as 0 is the starting point for us i.e Bangalore
    return random_route


In [44]:
temp_route = createRandomRoute()

In [45]:
temp_route

[3, 10, 6, 4, 9, 8, 5, 7, 1, 2]

## fitnessFunction :: Function to compute the route cost using the already defined path 

Fitness function here is the distance travelled if a particular path is taken.

    - Step 1: Use the generated route
    - Step 2: Calculate the distance travelled.

In [46]:
def fitnessFunction(pvRouteMap):
    """This functions takes the route generated and returns the cost incurred."""
    traverseData    = distance_matrix.copy() # creating a copy of the original df
    sourcePoint     = 0 # defining a starting point
    stopsCovered    = 0 # setting the number of stops covered to 0
    routeCost       = 0 # setting the initial cost to 0
    pvRouteMap1 = pvRouteMap
    
    # initiate a while loop and calculate the cost for the whole path traversed and return the cost
    while(stopsCovered < len(pvRouteMap1)) :  
        routeCost = routeCost + traverseData.iloc[sourcePoint, pvRouteMap1[stopsCovered]]   
        """route cost is the sum of the cost incurred from travelling from one point to the next according
        to the route that was generated previously."""
        sourcePoint = pvRouteMap1[stopsCovered] # update the source point to the next point in the route
        stopsCovered = stopsCovered+1 # Adding 1 to the stopsCovered
#         print(sourcePoint)
    routeCost = routeCost + traverseData.iloc[pvRouteMap1[-1],0]

    return(routeCost)

In [47]:
fitnessFunction(temp_route)

16116

In [48]:
fitnessFunction(one_route)

13859

#### Pick one route randomly

In [49]:
one_route

[6, 2, 10, 9, 5, 4, 7, 3, 8, 1]

In [50]:
one_route_cities = []
for i in one_route:
    one_route_cities.append(cities_mapping[i])
    
one_route_cities = [starting_point] + one_route_cities + [starting_point]
print(one_route_cities)

['Bangalore', 'Mysore', 'Varanasi', 'Hyderabad', 'Mumbai', 'Chennai', 'Delhi', 'Agra', 'Udaipur', 'Kolkata', 'Jaipur', 'Bangalore']


In [51]:
print("Total distance travelled to cover the route of {} is {}".format(" => ".join( one_route_cities ),fitnessFunction(one_route)))


Total distance travelled to cover the route of Bangalore => Mysore => Varanasi => Hyderabad => Mumbai => Chennai => Delhi => Agra => Udaipur => Kolkata => Jaipur => Bangalore is 13859


# Function to create Initial (random) Population 

In [52]:
def initialPopCost(ninitpop = 10):
    """This function generates new routes and calculates the cost of the route.
    returns a dictionary of routes and the cost."""
    np.random.seed(seed=42)
    intial_cost = 0
    routeCost = []
    routes = [createRandomRoute() for i in range(0, ninitpop)]
    # print(routes)
    routes_Cost = [fitnessFunction(i) for i in routes]
    # print(routes_Cost)
    routes_DF = pd.DataFrame([routes,routes_Cost]).transpose()
    routes_DF.columns = ['Route','Cost']
    return(routes_DF)

#### Example

In [53]:
initial_pop_cost = initialPopCost(20)
sorted_init_pop = initial_pop_cost.sort_values(['Cost'])

In [54]:
initial_pop_cost.head()

,Route,Cost
0,"[2, 3, 6, 9, 8, 4, 5, 7, 1, 10]",15855
1,"[3, 5, 2, 7, 4, 8, 10, 9, 1, 6]",13628
2,"[3, 7, 8, 5, 10, 1, 4, 6, 2, 9]",14501
3,"[5, 10, 7, 2, 4, 6, 3, 8, 9, 1]",14931
4,"[4, 2, 9, 1, 6, 8, 10, 3, 5, 7]",18602


In [55]:
sorted_init_pop.head()

,Route,Cost
11,"[5, 2, 3, 8, 10, 4, 7, 1, 9, 6]",11090
15,"[10, 9, 6, 4, 7, 5, 3, 1, 2, 8]",12715
10,"[7, 1, 3, 5, 10, 8, 4, 2, 6, 9]",13105
9,"[2, 7, 9, 3, 1, 10, 6, 8, 4, 5]",13168
1,"[3, 5, 2, 7, 4, 8, 10, 9, 1, 6]",13628


## Partially Mapped Crossover

###### - Implemented PMX conceptualized by goldberg

In [28]:
def crossOverFunction(parent1, parent2,crossover_factor_start_pos=2,
                             crossover_factor_end_pos=6):
    
    """This function implements the partially mapped crossover by goldeberg (https://www.hindawi.com/journals/cin/2017/7430125/)
    
    inputs: 2 parent solutions, crossover start position, crossover end position.
    crossover start position and  crossover end position are randomly generated in the solution.
    
    output: 2 child solutions, i.e the crossedover solutions.
    Example :
    
        Inputs:
            p1 = [5,7,2,4,8,9,3,6]
            p2 = [8,6,9,3,2,5,7,4]
            crossover_start_pos 2 (randomly selected)
            crossover_end_pos 6 (randomly selected)
        Internals:
            The mapping created is [(2, 9), (4, 3), (8, 2), (9, 5)]
            Child1 Intermediate [9, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [2, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [8, 7, 9, 3, 2, 5, 4, 6]
            Child1 final [8, 7, 9, 3, 2, 5, 4, 6]
            Child2 original [8, 6, 2, 4, 8, 9, 7, 4]
            Child2 Intermediate [2, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [9, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [5, 6, 2, 4, 8, 9, 7, 3]
            Child2 final [5, 6, 2, 4, 8, 9, 7, 3]
        
        Output:
            Child: [8, 7, 9, 3, 2, 5, 4, 6]
            Child: [5, 6, 2, 4, 8, 9, 7, 3].
    """
    
    # Generating two indices for crossover process
    indexes_for_crossover = random.sample(set(range(len(parent1))), 2)
    
    # selecting crossover starting and ending index
    crossover_factor_start_pos,crossover_factor_end_pos = min(indexes_for_crossover),max(indexes_for_crossover)
    
    # print("start end")
    # print(crossover_factor_start_pos,"    ",crossover_factor_end_pos)
    # print("\n")
    
    ## generate child 1
        
    child1 = parent1[0:crossover_factor_start_pos]+\
    parent2[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent1[crossover_factor_end_pos:]
    # print("Child1 original {}".format(child1))
    
    ## generate child 2
    child2 = parent2[0:crossover_factor_start_pos] +\
    parent1[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent2[crossover_factor_end_pos:]
    # print("Child2 original {}".format(child2))
    
    ## Create mappings for the Genes within those indices from two parents which underwent crossover
    mpping = list(zip(parent1[crossover_factor_start_pos:crossover_factor_end_pos], # create tuple of mappings
                      parent2[crossover_factor_start_pos:crossover_factor_end_pos]))
    # print("mapping")
    # print(mpping)
    
    # print("\n")
    # run until all the nodes in the routes are unique
    while len(np.unique(child1)) != len(child1):
        # extracting genes from child1 which did not undergo crossover
        child1_part = child1[:crossover_factor_start_pos]+child1[crossover_factor_end_pos:]
        # print("child1_part ",child1_part)
        for i in child1_part:
            for j in mpping:
                if i == j[1]: #j[1] - Parent 2 window elements                 
                    child1_part[child1_part.index(i)] = j[0] #j[0] - Parent 1 window elements
                    
        child1 = child1_part[:crossover_factor_start_pos]+ child1[crossover_factor_start_pos:crossover_factor_end_pos]+ child1_part[crossover_factor_start_pos:]
        
        # print("Child1 Intermediate {}".format(child1))
    
    # print("Child1 final {}".format(child1))

# run until all the nodes in the route are unique
    while len(np.unique(child2)) != len(child2):
        
        # extracting genes from child2 which did not undergo crossover
        child2_part = child2[:crossover_factor_start_pos]+child2[crossover_factor_end_pos:]
        # print("\n")
        # print("child2_part ",child2_part)
        for i in child2_part:
            for j in mpping:
                if i == j[0]:
                    child2_part[child2_part.index(i)] = j[1]
        child2 = child2_part[:crossover_factor_start_pos]+ child2[crossover_factor_start_pos:crossover_factor_end_pos]+ child2_part[crossover_factor_start_pos:]
        # print("Child2 Intermediate {}".format(child2))
    # print("Child2 final {}".format(child2))
    # print("\n")
    return( child1,child2)

In [29]:
p1 = [5,7,2,4,8,9,3,6]

In [30]:
p2 = [8,6,9,3,2,5,7,4]

In [31]:
for i in zip([p1,p2],crossOverFunction(p1,p2)):
    print ("Parent: {}  : Child: {}".format(i[0],i[1]))

Parent: [5, 7, 2, 4, 8, 9, 3, 6]  : Child: [5, 6, 9, 3, 2, 8, 4, 7]
Parent: [8, 6, 9, 3, 2, 5, 7, 4]  : Child: [9, 7, 2, 4, 8, 5, 6, 3]


#### We can look at the positions and think of one good route , we will see if our algorithm performs better than this route

In [27]:
better_route = [5, 10, 8, 2, 4, 7, 1, 3, 9, 6] # one of the better routes
fitnessFunction(better_route)

6815

Total distance travelled to cover the final route of 
 Bangalore => Chennai => Hyderabad => Kolkata => Varanasi => Delhi => Agra => Jaipur => Udaipur => Mumbai => Mysore => Bangalore is 6815


##  mutateRoute:: this function will generate mutated path where we define mutation policy.

In [28]:
def mutateRoute(initPath, mutateFactor):
    """ This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string(route here) is split and the two parts are swapped.

    
    Mutation
    Step 1: Pick a parent chromosome
    Step 2: Randomly pick an index (position in the parent chromosome)
    Step 3: Swap the values before and after the index to generate the mutated chromosome (child).
        
    Example: 
        input to the function:        
            parent = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10]
            index = 3 (randomly generate the index)
            
        with in function
            temp1 = [4, 6, 2]
            temp2 = [8, 3, 5, 7, 1, 9, 10]
        
        output:
            child  = [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

    Implemetation:
        getMutatedPath(initPath = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10], mutateFactor = 3)
        output: [8, 3, 5, 7, 1, 9, 10, 4, 6, 2].
    """
        
    try:
        
        temp1 = initPath[0:mutateFactor] # Part 1 
        
        temp2 = initPath[mutateFactor:len(initPath)] # part 2
        
        newPath = temp2 + temp1 # Swapping Part2 + Part1
    
    except:
        
        # This loop is executed when you enter more than 1 value for mutate factor 
        
        temp1 = initPath[0:max(mutateFactor)] # Part 1
        
        temp2 = initPath[max(mutateFactor):] # part 2
        
        newPath = temp2 + temp1 # Swapping Part2 + Part1
    
    return(newPath)

#### Generate a sample path and mutate that path 

In [29]:
temp_route = createRandomRoute()

In [30]:
temp_route

[9, 5, 4, 8, 1, 2, 3, 6, 10, 7]

In [31]:
mutateRoute(temp_route,2)

[4, 8, 1, 2, 3, 6, 10, 7, 9, 5]

In [32]:
fitnessFunction(temp_route)

15333

In [33]:
def genetic_algorithm(sorted_current_generation, no_of_chromosomes = 100, mutation_factor=0.1, xover_factor = 0.3):
         
    """This functions takes in a population and performs crossover on the top_percentage records.
    Output is a set of children after the crossover and mutation operation."""
    
    nextgen_children = [] 
    
    # Get the elite population by passing the xover factor
    # Get parent (Randomly generate two parents from the subset)

    xover_pool = int(round(no_of_chromosomes * xover_factor))
    no_of_loops = int((no_of_chromosomes // 2) )
    
    print("#####################################################################################")
    print ('Starting crossover')
    for i in range(0,no_of_loops):
        parent_index1 = random.choice(range(0,xover_pool)) # random.choice - Return a random element from the sequence
        p1= sorted_current_generation.iloc[parent_index1,0]
        print("parent 1")
        print(p1)
        parent_index2 = random.choice(range(0,xover_pool))
        p2= sorted_current_generation.iloc[parent_index2,0]
        print("parent 2")
        print(p2)

        c1, c2 = crossOverFunction(p1,p2)
        print("child 1")
        print(c1)
        print("child 2")
        print(c2)
        
        # Appending all the crossover children to the nextgen_children list
        nextgen_children.extend([c1,c2])
    
    print("nextgen_children")
    print(nextgen_children)
    
    # Evaluating the fitness of the crossover children
    nextgen_costs = [fitnessFunction(i) for i in nextgen_children]
    
    # Creating dataframe with routes and cost of the child
    next_generation = pd.DataFrame([nextgen_children,nextgen_costs],).transpose() 
    
    # Renaming the column names
    next_generation.columns = ['Route','Cost']
    
    print("#####################################################################################")
    print('Starting Mutation')
 
    for i in range(0, no_of_chromosomes):
        mutation_prob = random.random() # Return the next random floating point number in the range [0.0, 1.0)
        if mutation_prob < mutation_factor: 
            mutated_child = mutateRoute(next_generation.iloc[i,0] , 2)
            print("mutated_child",i+1)
            print(mutated_child)
            next_generation.iloc[i,0] = mutated_child
            next_generation.iloc[i,1] = fitnessFunction(next_generation.iloc[i,0])
    return (next_generation)

In [34]:
def perform_genetic_algorithm(no_of_iterations = 30, no_of_chromosomes = 100, mutation_factor=0.1, xover_factor = 0.3):
     
    """ This function runs the whole GA process, i.e.
        1. Generate initial population
        2. Crossover
        3. Mutation
    and returns the best solution.
    
    Inputs : overall GA runs, initial population size, mutation_factor, xover_factor
    
    """
    
    best_solution = []
    best_cost = 99999
    current_generation = initialPopCost(no_of_chromosomes)
    sorted_current_generation = current_generation.sort_values('Cost')
    
    for i in range(0, no_of_iterations) :
        next_generation = genetic_algorithm(sorted_current_generation,no_of_chromosomes,mutation_factor, xover_factor)
        sorted_next_generation = next_generation.sort_values('Cost')
        print ("Completed {} runs of GA.".format(i + 1))
        print ("below are the best chromosomes:")
        print(sorted_next_generation.head(3))
        
        sorted_current_generation = sorted_next_generation
        if best_cost > sorted_next_generation.iloc[0,1] :
            best_cost = sorted_next_generation.iloc[0,1]
            best_solution = sorted_next_generation.iloc[0,0]

        
        final_route = []
        for i in best_solution:
            final_route.append(cities_mapping[i])
        final_route = [starting_point] + final_route + [starting_point]
        print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))
               
    return (best_solution, best_cost)

In [35]:
best_solution, best_cost = perform_genetic_algorithm()

#####################################################################################
Starting crossover
parent 1
[5, 6, 4, 7, 3, 9, 10, 8, 2, 1]
parent 2
[6, 4, 5, 10, 1, 8, 2, 7, 9, 3]
child 1
[5, 6, 4, 8, 3, 10, 2, 7, 9, 1]
child 2
[6, 4, 5, 9, 1, 7, 10, 8, 2, 3]
parent 1
[9, 5, 8, 2, 4, 10, 6, 1, 3, 7]
parent 2
[1, 10, 9, 6, 5, 8, 4, 7, 2, 3]
child 1
[8, 10, 9, 6, 4, 5, 2, 1, 3, 7]
child 2
[1, 5, 8, 2, 10, 9, 4, 7, 6, 3]
parent 1
[6, 10, 4, 7, 3, 1, 2, 5, 9, 8]
parent 2
[5, 6, 7, 2, 3, 4, 1, 8, 9, 10]
child 1
[6, 10, 4, 2, 3, 1, 7, 5, 9, 8]
child 2
[5, 6, 2, 7, 3, 4, 1, 8, 9, 10]
parent 1
[10, 9, 8, 2, 4, 1, 3, 7, 5, 6]
parent 2
[2, 5, 10, 1, 9, 4, 3, 7, 8, 6]
child 1
[10, 2, 8, 1, 9, 4, 3, 7, 5, 6]
child 2
[9, 5, 10, 2, 4, 1, 3, 7, 8, 6]
parent 1
[6, 10, 4, 7, 3, 1, 2, 5, 9, 8]
parent 2
[7, 8, 5, 9, 3, 2, 4, 1, 10, 6]
child 1
[6, 10, 5, 9, 3, 2, 1, 4, 7, 8]
child 2
[9, 8, 4, 7, 3, 1, 5, 2, 10, 6]
parent 1
[7, 8, 5, 9, 3, 2, 4, 1, 10, 6]
parent 2
[10, 6, 4, 8, 5, 7, 1, 2, 3, 9]
chi

#####################################################################################
Starting Mutation
mutated_child 1
[2, 4, 1, 10, 3, 7, 9, 6, 8, 5]
mutated_child 14
[1, 2, 4, 7, 8, 9, 6, 3, 5, 10]
mutated_child 43
[1, 2, 4, 7, 6, 8, 10, 5, 9, 3]
mutated_child 44
[1, 2, 6, 10, 8, 7, 4, 5, 9, 3]
mutated_child 61
[7, 4, 5, 6, 1, 9, 3, 10, 8, 2]
mutated_child 67
[1, 7, 3, 9, 6, 2, 8, 5, 10, 4]
mutated_child 76
[7, 2, 6, 10, 8, 1, 9, 4, 3, 5]
mutated_child 77
[10, 7, 3, 1, 2, 6, 9, 5, 4, 8]
mutated_child 81
[2, 7, 3, 4, 1, 8, 9, 5, 10, 6]
mutated_child 89
[1, 7, 6, 10, 8, 2, 4, 5, 9, 3]
Completed 2 runs of GA.
below are the best chromosomes:
                              Route  Cost
92  [9, 3, 1, 10, 7, 4, 2, 8, 5, 6]  8993
93  [9, 3, 1, 10, 7, 4, 2, 8, 5, 6]  8993
90  [10, 2, 7, 4, 3, 9, 1, 8, 5, 6]  9033
Total distance travelled to cover the final route of 
 Bangalore => Mumbai => Udaipur => Jaipur => Hyderabad => Agra => Delhi => Varanasi => Kolkata => Chennai => Mysore => Bangalore 

#####################################################################################
Starting Mutation
mutated_child 10
[1, 10, 7, 4, 2, 8, 5, 6, 9, 3]
mutated_child 12
[7, 10, 8, 3, 2, 9, 6, 5, 1, 4]
mutated_child 14
[7, 2, 4, 8, 1, 9, 6, 5, 10, 3]
mutated_child 30
[1, 7, 8, 4, 2, 9, 6, 3, 5, 10]
mutated_child 37
[2, 8, 9, 4, 1, 7, 3, 5, 10, 6]
mutated_child 38
[1, 10, 5, 7, 2, 4, 6, 8, 9, 3]
mutated_child 39
[2, 8, 4, 1, 3, 7, 9, 10, 5, 6]
mutated_child 54
[1, 2, 4, 7, 10, 8, 6, 5, 9, 3]
mutated_child 63
[7, 2, 4, 1, 10, 8, 6, 3, 9, 5]
mutated_child 66
[3, 4, 1, 8, 9, 5, 10, 2, 6, 7]
mutated_child 86
[2, 5, 10, 1, 9, 3, 4, 6, 7, 8]
mutated_child 95
[2, 8, 4, 7, 3, 5, 9, 6, 1, 10]
Completed 3 runs of GA.
below are the best chromosomes:
                              Route  Cost
69  [9, 3, 1, 8, 7, 4, 2, 10, 5, 6]  8488
38  [2, 8, 4, 1, 3, 7, 9, 10, 5, 6]  8612
93  [5, 6, 7, 4, 3, 1, 2, 8, 10, 9]  8903
Total distance travelled to cover the final route of 
 Bangalore => Mumbai => Udaipu

parent 1
[9, 6, 8, 2, 3, 4, 1, 7, 10, 5]
parent 2
[5, 6, 7, 4, 3, 1, 2, 8, 10, 9]
child 1
[9, 6, 8, 4, 3, 2, 1, 7, 10, 5]
child 2
[5, 6, 7, 2, 3, 1, 4, 8, 10, 9]
parent 1
[6, 5, 7, 1, 3, 4, 2, 8, 10, 9]
parent 2
[8, 2, 3, 4, 1, 7, 10, 5, 9, 6]
child 1
[6, 8, 3, 4, 1, 7, 10, 5, 2, 9]
child 2
[5, 10, 7, 1, 3, 4, 2, 8, 9, 6]
parent 1
[2, 8, 4, 1, 3, 7, 9, 5, 10, 6]
parent 2
[2, 8, 4, 1, 3, 7, 9, 10, 5, 6]
child 1
[2, 8, 4, 1, 3, 7, 9, 10, 5, 6]
child 2
[2, 8, 4, 1, 3, 7, 9, 5, 10, 6]
parent 1
[8, 2, 3, 4, 1, 7, 10, 5, 9, 6]
parent 2
[2, 8, 4, 1, 3, 7, 9, 5, 10, 6]
child 1
[2, 8, 4, 1, 3, 7, 9, 5, 10, 6]
child 2
[8, 2, 3, 4, 1, 7, 10, 5, 9, 6]
parent 1
[2, 8, 4, 1, 3, 7, 9, 5, 10, 6]
parent 2
[9, 3, 1, 2, 7, 4, 10, 8, 5, 6]
child 1
[2, 5, 4, 1, 3, 7, 10, 8, 9, 6]
child 2
[10, 3, 1, 2, 7, 4, 9, 5, 8, 6]
parent 1
[9, 6, 8, 2, 3, 4, 1, 7, 10, 5]
parent 2
[6, 5, 7, 2, 4, 8, 1, 3, 9, 10]
child 1
[9, 6, 7, 2, 4, 8, 1, 3, 10, 5]
child 2
[6, 5, 8, 2, 3, 4, 1, 7, 9, 10]
parent 1
[2, 8, 4, 1, 3, 7, 

#####################################################################################
Starting Mutation
mutated_child 4
[7, 4, 3, 1, 10, 5, 9, 6, 8, 2]
mutated_child 7
[7, 2, 4, 1, 3, 8, 5, 6, 9, 10]
mutated_child 10
[3, 4, 2, 8, 5, 6, 10, 9, 7, 1]
mutated_child 22
[7, 2, 3, 4, 1, 8, 9, 5, 6, 10]
mutated_child 75
[7, 3, 1, 4, 2, 8, 10, 9, 5, 6]
mutated_child 77
[3, 1, 7, 4, 2, 10, 5, 6, 9, 8]
mutated_child 88
[7, 1, 9, 3, 2, 8, 5, 6, 10, 4]
Completed 6 runs of GA.
below are the best chromosomes:
                              Route  Cost
32  [6, 9, 3, 1, 7, 4, 2, 8, 5, 10]  7214
79  [5, 10, 3, 4, 1, 7, 2, 8, 9, 6]  8073
77  [9, 10, 7, 4, 1, 3, 2, 8, 5, 6]  8075
Total distance travelled to cover the final route of 
 Bangalore => Mysore => Mumbai => Udaipur => Jaipur => Agra => Delhi => Varanasi => Kolkata => Chennai => Hyderabad => Bangalore is 7214
#####################################################################################
Starting crossover
parent 1
[2, 8, 4, 1, 3, 7, 9, 10, 

...

#####################################################################################
Starting Mutation
mutated_child 7
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 52
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 54
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 56
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 57
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 58
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 70
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 81
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
Completed 26 runs of GA.
below are the best chromosomes:
                              Route  Cost
0   [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
72  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
71  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Mumbai => Udaipur => Jaipur => Delhi => Agra => Varanasi => Kolkata => Chennai => Mysore => Bangalore is 6543
#####################################################################################
Sta

#####################################################################################
Starting Mutation
mutated_child 37
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 44
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 57
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 86
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 94
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 96
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 99
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
Completed 27 runs of GA.
below are the best chromosomes:
                              Route  Cost
0   [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
70  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
69  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Mumbai => Udaipur => Jaipur => Delhi => Agra => Varanasi => Kolkata => Chennai => Mysore => Bangalore is 6543
#####################################################################################
Starting crossover
parent 1
[10, 9, 3, 1, 4, 7, 2, 

mutated_child 1
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 3
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 6
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 15
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 22
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 28
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 33
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 34
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 39
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 47
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 68
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 72
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 77
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 79
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 80
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 92
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 94
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
Completed 28 runs of GA.
below are the best chromosomes:
                              Route  Cost
49  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
68  [10, 9, 3, 1, 4, 7, 2, 8,

child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 1
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
parent 2
[10, 9, 3, 1, 4, 7, 2, 8, 5, 6]
child 1
[10, 9, 3, 1, 4, 7, 

#####################################################################################
Starting Mutation
mutated_child 7
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 41
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 43
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 55
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 63
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 76
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
mutated_child 81
[3, 1, 4, 7, 2, 8, 5, 6, 10, 9]
Completed 30 runs of GA.
below are the best chromosomes:
                              Route  Cost
0   [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
71  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
70  [10, 9, 3, 1, 4, 7, 2, 8, 5, 6]  6543
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Mumbai => Udaipur => Jaipur => Delhi => Agra => Varanasi => Kolkata => Chennai => Mysore => Bangalore is 6543


In [36]:
final_route = []
for i in best_solution:
    final_route.append(cities_mapping[i])
final_route = [starting_point] + final_route + [starting_point]
print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))


Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Mumbai => Udaipur => Jaipur => Delhi => Agra => Varanasi => Kolkata => Chennai => Mysore => Bangalore is 6543
